In [16]:
# import

from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd
import json
import time

# using headers to identify the scraper as a real user

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.157 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.5",
    "Referer": "https://google.com",
    "DNT": "1"
}

# staring date

date = 2011

#variable assignment

bestsellers_winners = []

#dictionary creation

genre_mapping = {
    "fiction": "fiction",
    "mystery-thriller": "mystery-thriller",
    "fantasy": "fantasy",
    "romance": "romance",
    "science-fiction": "science-fiction",
    "horror": "horror",
    "humor": "humor",
    "nonfiction": "nonfiction",
    "memoir-autobiography": "memoir-autobiography",
    "history-biography": "history-biography",
    "poetry": "poetry",
    "young-adult-fiction": "young-adult-fiction",
    "young-adult-fantasy": "young-adult-fantasy",
    "childrens": "childrens"
}

#creating a for loop to iterate through every year and every genre

for genre in genre_mapping.keys():
    current_date = date
    while current_date != 2023:
        genre_value = genre_mapping[genre]
        
        #the url I want to scrape
        url = f"https://www.goodreads.com/choiceawards/best-{genre_value}-books-{current_date}"
        
        #requesting the url and save the response to the response variable
        response = requests.get(url, headers=headers)
        
        #save the content of response to html
        html = response.content
        
        #creating a Beautiful Soup object named soup
        soup = bs(html, "lxml")
        
        #in order not to banned from the site
        time.sleep(1)

        #dictionary creation
        bestseller = {}
        
        #filling up the dictionary
        bestseller["genre"] = genre
        bestseller["year"] = current_date
        
        #extract data from soup
        votes_element = soup.find("strong", class_="uitext result")
        bestseller["votes"] = votes_element.get_text(strip=True)[-13:-6].replace(",", "") if votes_element else ""

        votestotal_element = soup.find("div", class_="greyText gcaNumVotes")
        bestseller["votestotal"] = votestotal_element.get_text(strip=True).replace(",", "").replace("\nvotes total", "") if votestotal_element else ""
        

        #the following information can only be found using the links on the page, so I searched for them and iterated through them
        bestseller_link = soup.find('a', class_='pollAnswer__bookLink')
        if bestseller_link:
            
            #the urls I want to scrape
            bestseller_url = f"https://www.goodreads.com{bestseller_link['href']}"
            bestseller["url"] = bestseller_url

            #requesting the url and save the response to the response variable
            bestseller_url_response = requests.get(bestseller_url, headers=headers)
            
            #creating a Beautiful Soup object named bestseller_soup
            bestseller_soup = bs(bestseller_url_response.content, "lxml")

            #extract data from soup
            title_element = bestseller_soup.find('meta', property='og:title')
            if title_element:
                bestseller["title"] = title_element.get('content')
            else:
                bestseller["title"] = "Not available"
            
            author = bestseller_soup.find('span', {'data-testid': 'name'})
            if author:
                bestseller["author"] = author.text.strip()
            else:
                bestseller["author"] = "Not available"
            
            reviews = bestseller_soup.find("span", class_="u-dot-before")
            bestseller["reviews"] = reviews.get_text()[-15:-8].replace(",", "") if reviews else "Not available"

            script_tag = bestseller_soup.find('script', type='application/ld+json')
            if script_tag:
                script_data = json.loads(script_tag.string)
                bestseller["ratings"] = script_data.get('aggregateRating', {}).get('ratingCount')
            else:
                bestseller["ratings"] = "Not available"
                    
            script_element = bestseller_soup.find('script', {'type': 'application/ld+json'})
            if script_element:
                json_data = json.loads(script_element.string)
                bestseller["avg_ratings"] = json_data['aggregateRating']['ratingValue']
            else:
                bestseller["avg_ratings"] = "Not available"
            
            
            script_element = bestseller_soup.find("script", type="application/ld+json")
            if script_element:
                script_content = script_element.string
                data = json.loads(script_content)
                bestseller["numberOfPages"] = data.get("numberOfPages", "Not available")
            else:
                bestseller["numberOfPages"] = "Not available"
                    
            nbOfBooks = bestseller_soup.select_one('h4 + span.Text.Text__body3.Text__subdued')
            pattern = r"books[\d,]+[.\dkK]*followers"
            nbOfBooks = nbOfBooks.get_text(strip=True)
            bestseller["nbOfBooks"] = re.sub(pattern, "", nbOfBooks) if nbOfBooks else "Not available"
            
            followers = bestseller_soup.select_one('h4 + span.Text.Text__body3.Text__subdued').find("span", class_="u-dot-before")
            bestseller["followers"] = followers.get_text(strip=True).replace("kfollowers", "").replace("followers", "").replace(",", ".") if followers else "Not available"
            
            
            pattern = r'"bookFormat":"(.*?)"'
            match = re.search(pattern, str(bestseller_url_response.content))
            bestseller["format"] = match.group(1) if match else "Not available"

            pattern = r'"inLanguage":"(.*?)"'
            match = re.search(pattern, str(bestseller_url_response.content))
            bestseller["language"] = match.group(1) if match else "Not available"

            #addig the dictionary bestseller to the list bestsellers_winners
            bestsellers_winners.append(bestseller)

        #to move with the date after finishing the loop
        current_date += 1

#creating a pandas DataFrame from the list bestsellers_winners      
bestsellers_winners_df = pd.DataFrame(bestsellers_winners)
bestsellers_winners_df

#saving the data frame
bestsellers_winners_df.to_csv("/home/biologist/bestsellers_winners.csv", index=False)

In [17]:
bestsellers_winners_df

,author,avg_ratings,followers,format,genre,language,nbOfBooks,numberOfPages,ratings,reviews,title,url,votes,votestotal,year
0,Haruki Murakami,3.94,111,Hardcover,fiction,English,491,925,246304,21419,"1Q84 (1Q84, #1-3)",https://www.goodreads.com/book/show/10357575-1...,4107,31097,2011
1,J.K. Rowling,3.30,221,Hardcover,fiction,English,782,503,323730,32290,The Casual Vacancy,https://www.goodreads.com/book/show/13497818-t...,11525,74417,2012
2,Khaled Hosseini,4.07,151,Hardcover,fiction,English,39,404,355020,31575,And the Mountains Echoed,https://www.goodreads.com/book/show/16115612-a...,24686,137340,2013
3,Rainbow Rowell,3.54,139,Hardcover,fiction,English,149,320,164188,19895,Landline,https://www.goodreads.com/book/show/18081809-l...,46154,234858,2014
4,Harper Lee,3.31,13,Hardcover,fiction,English,72,278,263560,31933,Go Set a Watchman,https://www.goodreads.com/book/show/24817626-g...,31093,190758,2015
5,Liane Moriarty,3.59,61.2,Hardcover,fiction,English,52,432,283426,22022,Truly Madly Guilty,https://www.goodreads.com/book/show/26247008-t...,30154,237844,2016
6,Celeste Ng,4.08,87,Hardcover,fiction,English,15,338,1082060,70889,Little Fires Everywhere,https://www.goodreads.com/book/show/34273236-l...,39077,271245,2017
7,Jojo Moyes,4.12,58.4,Hardcover,fiction,English,88,368,191007,14780,"Still Me (Me Before You, #3)",https://www.goodreads.com/book/show/35791968-s...,55300,332119,2018
8,Margaret Atwood,4.19,77.9,Hardcover,fiction,English,586,422,325927,29644,"The Testaments (The Handmaid's Tale, #2)",https://www.goodreads.com/book/show/42975172-t...,98291,345652,2019
9,Matt Haig,4.02,32.5,Hardcover,fiction,English,103,304,1391328,150341,The Midnight Library,https://www.goodreads.com/book/show/52578297-t...,72828,506978,2020
